In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
from data_loader import dataloader
user,ratings,movies = dataloader()
ratings = ratings.drop("timestamp",axis=1)
movie_index_by_id = {id: i for i, id in enumerate(movies["movie_id"])}


In [2]:
class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
            learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        # regularization parameter
        self.lam = lam
        # learning rate for gradient descent
        self.learning_rate = learning_rate
        # maximum number of iterations
        self.max_iter = max_iter
        # print results after print_every iterations
        self.print_every = print_every
        # user-based or item-based
        self.user_based = user_based
        # number of users, items, and ratings. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = 3901
        self.n_ratings = Y_data.shape[0]
        
        if Xinit is None: # new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: # from daved data
            self.W = Winit
            
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy()


    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users

        # if we want to normalize based on item, just switch first two columns of data
        else: # item bas
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        
        # take average
        L /= self.n_ratings
        # regularization, don't ever forget this 
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 
    def get_items_rated_by_user(self, user_id):
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
            
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            # gradient
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            # gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        # truncate if results are out of range [0, 5]
        if pred < 0:
            return 0 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((y_pred[i],i))
        
        return predicted_ratings
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE

In [17]:
ratings = ratings.values
print(ratings)

[[   1 1193    5]
 [   1  661    3]
 [   1  914    3]
 ...
 [6040  562    5]
 [6040 1096    4]
 [6040 1097    4]]


In [18]:
ratings1 = ratings.copy()

In [19]:
ratings[:,1]= [movie_index_by_id[movie_id] for movie_id in ratings1[:,1]]


In [20]:
print(ratings)
a= np.max(ratings[:,1])

print(a)

[[   1 1176    5]
 [   1  655    3]
 [   1  902    3]
 ...
 [6040  558    5]
 [6040 1080    4]
 [6040 1081    4]]
3882


In [21]:
user_prof1 = pd.read_csv('u_dict.csv')
print(user_prof1.dtypes)
user_prof1 = user_prof1.values
print(user_prof1.shape)

user_id      int64
genres       int64
weight     float64
dtype: object
(93887, 3)


In [22]:
print(ratings[:, 0])
ratings[:, 0] -= 1
print(ratings[:, :2])
ratings = np.concatenate((ratings,user_prof1), axis = 0)

from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print(rate_train.shape)

print(len(rate_train))

rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, max_iter = 12, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

[   1    1    1 ... 6040 6040 6040]
[[   0 1176]
 [   0  655]
 [   0  902]
 ...
 [6039  558]
 [6039 1080]
 [6039 1081]]
(733044, 3)
733044
iter = 2 , loss = 7.021324669671378 , RMSE train = 1.1584189760494596
iter = 4 , loss = 4.602575130357182 , RMSE train = 1.0470293411176907
iter = 6 , loss = 3.119927512023703 , RMSE train = 1.025147884069602
iter = 8 , loss = 2.1820184782043723 , RMSE train = 1.021412211141571
iter = 10 , loss = 1.58361822187621 , RMSE train = 1.0207816492249953
iter = 12 , loss = 1.2009603038498629 , RMSE train = 1.0206748325295039

Item-based MF, RMSE = 1.0299906076064351


In [23]:

z = movie_index_by_id.copy()
z=list(z.items())
print(z[0][1])

0


In [44]:
a = rs.pred_for_user(1999)
a.sort(reverse= True)
for i in range(10):
    print(a[i][0],"-",movies.iloc[z[ a[i][1]][1],1],"-",movies.iloc[z[a[i][1]][1],2])

3.7462095662261605 - And the Ship Sails On (E la nave va) (1984) - Comedy|War
3.7435702182779003 - Children of the Corn IV: The Gathering (1996) - Horror
3.7419334182396486 - Waterworld (1995) - Action|Adventure
3.741487303005245 - Tingler, The (1959) - Horror
3.7403207517018493 - Bandit Queen (1994) - Drama
3.740063557486023 - Guardian, The (1990) - Horror|Thriller
3.739932407299352 - Against All Odds (1984) - Romance
3.739512920869 - Man on the Moon (1999) - Comedy|Drama
3.739430335168624 - Careful (1992) - Comedy
3.7390149165243765 - All About My Mother (Todo Sobre Mi Madre) (1999) - Comedy|Drama


In [38]:
import pickle
filename = 'rs_mf.sav'
pickle.dump(rs,open(filename,'wb'))

In [3]:
import pickle
rs1 = pickle.load((open('rs_mf.sav','rb')))
z = movie_index_by_id.copy()
z=list(z.items())
print(z[0][1])

0


In [7]:
b = rs1.pred_for_user(13)
b.sort(reverse= True)
for i in range(10):
    print(b[i][0],"-",movies.iloc[z[ b[i][1]][1],1],"-",movies.iloc[z[b[i][1]][1],2])


3.354835522909926 - And the Ship Sails On (E la nave va) (1984) - Comedy|War
3.35380752258602 - Children of the Corn IV: The Gathering (1996) - Horror
3.3536957666495555 - Guardian, The (1990) - Horror|Thriller
3.353313810456106 - Civil Action, A (1998) - Drama
3.3521061373961305 - Tingler, The (1959) - Horror
3.3510379684876606 - Airplane! (1980) - Comedy
3.35070846089654 - Bandit Queen (1994) - Drama
3.3506386431141566 - Against All Odds (1984) - Romance
3.3499295744093196 - Hedd Wyn (1992) - Drama
3.349470935369225 - Beauty (1998) - Drama


In [5]:
import time
start = time.time()
for i in range(1000):
    rs1.pred_for_user(i)
elapsed = (time.time() - start)/1000
print(elapsed)
    

0.012054246664047242
